In [1]:
import psycopg2
import numpy as np
import pandas as pd
import os
read_me=pd.DataFrame(columns = ['symbolname', 'file_name','date','counts', 'min_bid' , 'msx_ask'])
list_month={1:'January',2:'February', 3:'March', 4:'April', 5:'May', 6: 'June',
            7: 'July', 8: 'August', 9:'September', 10:'October', 11:'November', 12: 'December'}
#print ( f'{year}_{list_month[month]}')

In [2]:
def load_infosymbol():
    # Запрос infosymbol
    try:
        # connect to exist database
        connection = psycopg2.connect(
            host='35.184.100.252',
            user='postgres',
            password='jmgXK4xPXukY',
            database='data_db'
        )
        connection.autocommit = True

        cursor = connection.cursor()

        cursor.execute('''SELECT symbolnum, sql_server_time, symbolname, digets, pips, weaght FROM current_mt_db.infosymbol
        where symbolnum<100 order by symbolnum''')

        date_select = cursor.fetchall()
        infosymbols_array = np.asarray (date_select)
        infosymbols_ds = pd.DataFrame(infosymbols_array, columns = ['symbolnum','sql_server_time','symbolname', 'digets', 'pips', 'weaght'])
        return infosymbols_ds
    except Exception as _ex:
        print("[INFO] Error while working with PostgreSQL", _ex)
    finally:
        if connection:
            cursor.close()
            connection.close()
            print("[INFO] PostgreSQL connection closed")


#infosymbol

In [3]:
def digets_norm(df_exit, symbol_num):
    infosymbols_ds=load_infosymbol()
    diget=int(infosymbols_ds[infosymbols_ds['symbolnum']==symbol_num]['digets'])
    diget=pow(10,diget)

    df=df_exit.copy()
    df['open_bid']=df['open_bid']/diget
    df['high_bid']=df['high_bid']/diget
    df['low_bid']=df['low_bid']/diget
    df['close_bid']=df['close_bid']/diget
    df['open_ask']=df['open_ask']/diget
    df['high_ask']=df['high_ask']/diget
    df['low_ask']=df['low_ask']/diget
    df['close_ask']=df['close_ask']/diget

    return df

In [4]:
def save_as(df_exit, symbol_num, year, month):

    infosymbols_ds=load_infosymbol()

    diget=int(infosymbols_ds[infosymbols_ds['symbolnum']==symbol_num]['digets'])
    diget=pow(10,diget)

    df=df_exit.copy()
    df['open_bid']=df['open_bid']/diget
    df['high_bid']=df['high_bid']/diget
    df['low_bid']=df['low_bid']/diget
    df['close_bid']=df['close_bid']/diget
    df['open_ask']=df['open_ask']/diget
    df['high_ask']=df['high_ask']/diget
    df['low_ask']=df['low_ask']/diget
    df['close_ask']=df['close_ask']/diget

    if month<10:
        month_str=f'0{str(month)}'
    else:
        month_str=f'{str(month)}'

    #file_name=path
    katalog=f'{year}_{list_month[month]}'
    #symbolname=str(infosymbols_ds[infosymbols_ds['symbolnum']==symbol_num]['symbolname'])[5:11]
    file_name=f"{symbol_num}_{year}_{month_str}.csv"
    path = f"G:\\Общие диски\\Ci\\data_db_csv\\{katalog}\\"+file_name

    df.to_csv(
        path,  # шлях до файлу, в який робимо запис
        sep=';',  # роздільник колонок
        header=True,  # чи записувати імена колонок у файл (True - записувати, False - ні)
        index=False,  # записувати індекс у файл (True - записувати, False - ні)
    )
    return file_name

In [5]:
year=2021
month=12
#day=7
symbol_num=1

katalog=f'{year}_{list_month[month]}'
path = fr"G:\Общие диски\Ci\data_db_csv\{katalog}"

In [19]:
df = pd.read_csv('GoogleDriveTransfer_Ci_data_db_csv_2021_December_10_2021_12.csv', sep=';')
df

FileNotFoundError: [Errno 2] No such file or directory: 'GoogleDriveTransfer/Ci/data_db_csv/2023_January/29_2023_01.csv'

In [24]:
from google.auth.transport import requests
from googleapiclient import discovery
from google.oauth2 import service_account
from google.cloud import storage
from googleapiclient.http import MediaIoBaseDownload
import io

In [25]:
def get_client():
    #Google Storage
    credentials_storage = service_account.Credentials.from_service_account_file(
        filename='credentials.json',
        scopes=['https://www.googleapis.com/auth/devstorage.full_control'],
    )
    client = storage.Client(credentials=credentials_storage)
    bucket = client.get_bucket('cabinet_bucket')

    return bucket

In [26]:
bucket = get_client()

In [4]:
path = 'static/cms/js/dist/3.10.0/bundle.toolbar.min.js'
blob = bucket.blob(path)

In [5]:
blob_bytes_gen = blob.download_as_bytes()

In [47]:
def upload_file_csv(bucket, df, path):
    """Uploads a csv file to the bucket."""
    blob = bucket.blob(path)
    if blob.exists():
        df_read = pd.read_csv(io.StringIO(blob.download_as_text()), sep=';')
        df = df_read.append(df)
    fh = io.BytesIO()
    df.to_csv(fh, sep=';', header=True, index=False)
    blob = bucket.blob(path)
    blob.upload_from_string(fh.getvalue())
    fh.close()

In [40]:
df
path = 'GoogleDriveTransfer/Ci/data_db_csv/2023_January/29_2023_01.csv'
blob = bucket.blob(path)
if blob.exists():
    df_read = pd.read_csv(io.StringIO(blob.download_as_text()), sep=';')
    df = df_read.append(df)

upload_file_csv(bucket, df, path)


In [52]:
df = pd.read_csv(io.StringIO(blob.download_as_text()), sep=';')

In [27]:
path = 'GoogleDriveTransfer/Ci/data_db_csv/2023_January/29_2023_01.csv'
blob = bucket.blob(path)
if blob.exists():
    df = pd.read_csv(io.StringIO(blob.download_as_text()), sep=';')
df

,datetimesecond,open_bid,high_bid,low_bid,close_bid,open_ask,high_ask,low_ask,close_ask
0,2023-01-06 23:54:59,0.58879,0.58879,0.58879,0.58879,0.58912,0.58912,0.58912,0.58912


In [1]:
ls =  [['1 2023 1', 0], ['1 2023 2', 154004, 154004], ['2 2023 1', 77, 77], ['2 2023 2', 137347, 137347], ['3 2023 1', 22, 22], ['3 2023 2', 201701, 201701], ['4 2023 1', 1, 1], ['4 2023 2', 164404, 164404], ['5 2023 1', 30, 30], ['5 2023 2', 33531, 33531], ['6 2023 1', 109, 109], ['6 2023 2', 186603, 186603], ['7 2023 1', 65, 65], ['7 2023 2', 124153, 124153], ['8 2023 1', 1, 1], ['8 2023 2', 248414, 248414], ['9 2023 1', 1, 1], ['9 2023 2', 85760, 85760], ['10 2023 1', 1, 1], ['10 2023 2', 1713, 1713], ['11 2023 1', 0], ['11 2023 2', 213426, 213426], ['12 2023 1', 0], ['12 2023 2', 161172, 161172], ['13 2023 1', 1, 1], ['13 2023 2', 13418, 13418], ['14 2023 1', 0], ['14 2023 2', 166330, 166330], ['15 2023 1', 186, 186], ['15 2023 2', 154765, 154765], ['16 2023 1', 220, 220], ['16 2023 2', 172424, 172424], ['17 2023 1', 0], ['17 2023 2', 211222, 211222], ['18 2023 1', 0], ['18 2023 2', 121281, 121281], ['19 2023 1', 0], ['19 2023 2', 224579, 224579], ['20 2023 1', 0], ['20 2023 2', 229362, 229362], ['21 2023 1', 0], ['21 2023 2', 262413, 262413], ['22 2023 1', 0], ['22 2023 2', 219923, 219923], ['23 2023 1', 0], ['23 2023 2', 242502, 242502], ['24 2023 1', 0], ['24 2023 2', 183081, 183081], ['25 2023 1', 0], ['25 2023 2', 160788, 160788], ['26 2023 1', 0], ['26 2023 2', 160251, 160251], ['27 2023 1', 0], ['27 2023 2', 145943, 145943], ['28 2023 1', 0], ['28 2023 2', 210583, 210583], ['29 2023 1', 0], ['29 2023 2', 169817, 169817], ['30 2023 1', 1, 1], ['30 2023 2', 25133, 25133], ['31 2023 1', 0], ['31 2023 2', 163213, 163213], ['32 2023 1', 0], ['32 2023 2', 228169, 228169], ['33 2023 1', 0], ['33 2023 2', 229855, 229855], ['34 2023 1', 0], ['34 2023 2', 225932, 225932], ['35 2023 1', 0], ['35 2023 2', 210704, 210704], ['36 2023 1', 0], ['36 2023 2', 243146, 243146], ['37 2023 1', 0], ['37 2023 2', 0], ['38 2023 1', 0], ['38 2023 2', 275996, 275996], ['39 2023 1', 0], ['39 2023 2', 4, 4], ['40 2023 1', 1, 1], ['40 2023 2', 210570, 210570], ['41 2023 1', 0], ['41 2023 2', 246903, 246903], ['42 2023 1', 0], ['42 2023 2', 215643, 215643], ['43 2023 1', 189, 189], ['43 2023 2', 176012, 176012], ['44 2023 1', 1, 1], ['44 2023 2', 57895, 57895], ['45 2023 1', 0], ['45 2023 2', 177310, 177310], ['46 2023 1', 0], ['46 2023 2', 0], ['47 2023 1', 0], ['47 2023 2', 4, 4], ['48 2023 1', 0], ['48 2023 2', 129631, 129631], ['49 2023 1', 72, 72], ['49 2023 2', 132153, 132153], ['50 2023 1', 0], ['50 2023 2', 137140, 137140], ['51 2023 1', 1, 1], ['51 2023 2', 62774, 62774], ['52 2023 1', 26, 26], ['52 2023 2', 142700, 142700], ['53 2023 1', 95, 95], ['53 2023 2', 232457, 232457], ['54 2023 1', 1, 1], ['54 2023 2', 122796, 122796], ['55 2023 1', 1, 1], ['55 2023 2', 131827, 131827], ['56 2023 1', 1, 1], ['56 2023 2', 51352, 51352], ['57 2023 1', 1, 1], ['57 2023 2', 69771, 69771], ['58 2023 1', 1, 1], ['58 2023 2', 162932, 162932], ['59 2023 1', 1, 1], ['59 2023 2', 40720, 40720], ['60 2023 1', 1, 1], ['60 2023 2', 238088, 238088], ['61 2023 1', 1, 1], ['61 2023 2', 96510, 96510], ['62 2023 1', 0], ['62 2023 2', 0], ['63 2023 1', 1, 1], ['63 2023 2', 106488, 106488], ['64 2023 1', 1, 1], ['64 2023 2', 55298, 55298], ['65 2023 1', 1, 1], ['65 2023 2', 63105, 63105], ['66 2023 1', 1, 1], ['66 2023 2', 124243, 124243], ['67 2023 1', 0], ['67 2023 2', 4, 4], ['68 2023 1', 0], ['68 2023 2', 4, 4], ['69 2023 1', 0], ['69 2023 2', 4, 4], ['70 2023 1', 0], ['70 2023 2', 0], ['71 2023 1', 0], ['71 2023 2', 4, 4], ['72 2023 1', 0], ['72 2023 2', 128624, 128624], ['73 2023 1', 0], ['73 2023 2', 190444, 190444], ['74 2023 1', 0], ['74 2023 2', 222257, 222257], ['75 2023 1', 0], ['75 2023 2', 87013, 87013], ['76 2023 1', 0], ['76 2023 2', 112202, 112202], ['77 2023 1', 0], ['77 2023 2', 193994, 193994], ['78 2023 1', 0], ['78 2023 2', 159567, 159567], ['79 2023 1', 0], ['79 2023 2', 101453, 101453], ['80 2023 1', 0], ['80 2023 2', 90443, 90443], ['81 2023 1', 0], ['81 2023 2', 331801, 331801], ['82 2023 1', 0], ['82 2023 2', 171044, 171044], ['83 2023 1', 0], ['83 2023 2', 182040, 182040], ['84 2023 1', 0], ['84 2023 2', 269154, 269154], ['85 2023 1', 0], ['85 2023 2', 124832, 124832], ['86 2023 1', 0], ['86 2023 2', 137538, 137538], ['87 2023 1', 0], ['87 2023 2', 123361, 123361], ['88 2023 1', 5, 5], ['88 2023 2', 17542, 17542], ['89 2023 1', 0], ['89 2023 2', 226, 226], ['90 2023 1', 0], ['90 2023 2', 117449, 117449], ['91 2023 1', 0], ['91 2023 2', 124477, 124477], ['92 2023 1', 0], ['92 2023 2', 522, 522], ['93 2023 1', 0], ['93 2023 2', 107919, 107919], ['94 2023 1', 0], ['94 2023 2', 67694, 67694], ['95 2023 1', 1, 1], ['95 2023 2', 139612, 139612], ['96 2023 1', 3, 3], ['96 2023 2', 2541, 2541], ['97 2023 1', 0], ['97 2023 2', 0], ['98 2023 1', 0], ['98 2023 2', 0], ['99 2023 1', 0], ['99 2023 2', 0]]


In [50]:
df

,datetimesecond,open_bid,high_bid,low_bid,close_bid,open_ask,high_ask,low_ask,close_ask
0,2021-12-01 09:00:01,73.947,73.947,73.947,73.947,73.990,73.990,73.990,73.990
1,2021-12-01 09:00:05,73.951,73.951,73.951,73.951,73.993,73.993,73.993,73.993
2,2021-12-01 09:00:07,73.949,73.949,73.949,73.949,73.989,73.989,73.989,73.989
3,2021-12-01 09:00:10,73.945,73.945,73.945,73.945,73.990,73.990,73.990,73.990
4,2021-12-01 09:00:11,73.946,73.946,73.946,73.946,73.988,73.988,73.988,73.988
5,2021-12-01 09:00:14,73.947,73.947,73.947,73.947,73.989,73.989,73.989,73.989
6,2021-12-01 09:00:15,73.949,73.949,73.949,73.949,73.989,73.989,73.989,73.989
7,2021-12-01 09:00:19,73.953,73.953,73.953,73.953,73.997,73.997,73.997,73.997
8,2021-12-01 09:00:21,73.951,73.951,73.951,73.951,73.992,73.992,73.992,73.992
9,2021-12-01 09:00:24,73.954,73.954,73.954,73.954,73.997,73.997,73.997,73.997


In [45]:
df = df.head(5)
df['open_bid'] *= 10

C:\Users\Alex\AppData\Local\Temp/ipykernel_11568/2721496345.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['open_bid'] *= 10


In [48]:
upload_file_csv(bucket, df, path)

In [42]:
df_read

,datetimesecond,open_bid,high_bid,low_bid,close_bid,open_ask,high_ask,low_ask,close_ask
0,2021-12-01 09:00:01,73.947,73.947,73.947,73.947,73.990,73.990,73.990,73.990
1,2021-12-01 09:00:05,73.951,73.951,73.951,73.951,73.993,73.993,73.993,73.993
2,2021-12-01 09:00:07,73.949,73.949,73.949,73.949,73.989,73.989,73.989,73.989
3,2021-12-01 09:00:10,73.945,73.945,73.945,73.945,73.990,73.990,73.990,73.990
4,2021-12-01 09:00:11,73.946,73.946,73.946,73.946,73.988,73.988,73.988,73.988
5,2021-12-01 09:00:14,73.947,73.947,73.947,73.947,73.989,73.989,73.989,73.989
6,2021-12-01 09:00:15,73.949,73.949,73.949,73.949,73.989,73.989,73.989,73.989
7,2021-12-01 09:00:19,73.953,73.953,73.953,73.953,73.997,73.997,73.997,73.997
8,2021-12-01 09:00:21,73.951,73.951,73.951,73.951,73.992,73.992,73.992,73.992
9,2021-12-01 09:00:24,73.954,73.954,73.954,73.954,73.997,73.997,73.997,73.997


In [18]:
df = df.head(10)

In [6]:
df2 = df.tail(5).copy()
df2

,datetimesecond,open_bid,high_bid,low_bid,close_bid,open_ask,high_ask,low_ask,close_ask
347557,2021-12-31 09:11:34,74.521,74.521,74.521,74.521,74.687,74.687,74.687,74.687
347558,2021-12-31 09:13:14,74.513,74.513,74.513,74.513,74.695,74.695,74.695,74.695
347559,2021-12-31 09:13:54,74.512,74.512,74.512,74.512,74.694,74.694,74.694,74.694
347560,2021-12-31 09:13:56,74.513,74.513,74.513,74.513,74.695,74.695,74.695,74.695
347561,2021-12-31 09:14:10,74.512,74.577,74.512,74.512,74.512,74.694,74.599,74.694


In [3]:
df1 = df.head(5).copy()
df1

,datetimesecond,open_bid,high_bid,low_bid,close_bid,open_ask,high_ask,low_ask,close_ask
0,2021-12-01 09:00:01,73.947,73.947,73.947,73.947,73.990,73.990,73.990,73.990
1,2021-12-01 09:00:05,73.951,73.951,73.951,73.951,73.993,73.993,73.993,73.993
2,2021-12-01 09:00:07,73.949,73.949,73.949,73.949,73.989,73.989,73.989,73.989
3,2021-12-01 09:00:10,73.945,73.945,73.945,73.945,73.990,73.990,73.990,73.990
4,2021-12-01 09:00:11,73.946,73.946,73.946,73.946,73.988,73.988,73.988,73.988


In [7]:
df3 = pd.concat([df1, df2], ignore_index=True)
df3

,datetimesecond,open_bid,high_bid,low_bid,close_bid,open_ask,high_ask,low_ask,close_ask
0,2021-12-01 09:00:01,73.947,73.947,73.947,73.947,73.990,73.990,73.990,73.990
1,2021-12-01 09:00:05,73.951,73.951,73.951,73.951,73.993,73.993,73.993,73.993
2,2021-12-01 09:00:07,73.949,73.949,73.949,73.949,73.989,73.989,73.989,73.989
3,2021-12-01 09:00:10,73.945,73.945,73.945,73.945,73.990,73.990,73.990,73.990
4,2021-12-01 09:00:11,73.946,73.946,73.946,73.946,73.988,73.988,73.988,73.988
5,2021-12-31 09:11:34,74.521,74.521,74.521,74.521,74.687,74.687,74.687,74.687
6,2021-12-31 09:13:14,74.513,74.513,74.513,74.513,74.695,74.695,74.695,74.695
7,2021-12-31 09:13:54,74.512,74.512,74.512,74.512,74.694,74.694,74.694,74.694
8,2021-12-31 09:13:56,74.513,74.513,74.513,74.513,74.695,74.695,74.695,74.695
9,2021-12-31 09:14:10,74.512,74.577,74.512,74.512,74.512,74.694,74.599,74.694


In [62]:
df1['datetimesecond'].iloc[0] = '2021-12-02 09:00:01'

d:\it\pr\mypython\venv\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [4]:
# исключить из df1 строки, где datetimesecond в df есть
df2 = df1[~df1['datetimesecond'].isin(df['datetimesecond'])]

In [5]:
df2

,datetimesecond,open_bid,high_bid,low_bid,close_bid,open_ask,high_ask,low_ask,close_ask


In [59]:
df

,datetimesecond,open_bid,high_bid,low_bid,close_bid,open_ask,high_ask,low_ask,close_ask
0,2021-12-01 09:00:01,73.947,73.947,73.947,73.947,73.990,73.990,73.990,73.990
1,2021-12-01 09:00:05,73.951,73.951,73.951,73.951,73.993,73.993,73.993,73.993
2,2021-12-01 09:00:07,73.949,73.949,73.949,73.949,73.989,73.989,73.989,73.989
3,2021-12-01 09:00:10,73.945,73.945,73.945,73.945,73.990,73.990,73.990,73.990
4,2021-12-01 09:00:11,73.946,73.946,73.946,73.946,73.988,73.988,73.988,73.988
5,2021-12-01 09:00:14,73.947,73.947,73.947,73.947,73.989,73.989,73.989,73.989
6,2021-12-01 09:00:15,73.949,73.949,73.949,73.949,73.989,73.989,73.989,73.989
7,2021-12-01 09:00:19,73.953,73.953,73.953,73.953,73.997,73.997,73.997,73.997
8,2021-12-01 09:00:21,73.951,73.951,73.951,73.951,73.992,73.992,73.992,73.992
9,2021-12-01 09:00:24,73.954,73.954,73.954,73.954,73.997,73.997,73.997,73.997


In [9]:
connection = psycopg2.connect(
                host='127.0.0.1',
                user='postgres',
                password='11',
                database='test2'
            )
connection.autocommit = True

cursor = connection.cursor()

In [14]:
cursor.execute(f'''SELECT * FROM public.profile_app_appuser''')

In [15]:
date_select = cursor.fetchall()

In [16]:
my_array = np.asarray(date_select)

In [18]:
df = pd.DataFrame(my_array)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,4,pbkdf2_sha256$216000$MUPwu3T6EwEH$9iwq+MJdxKTM...,2022-12-29 20:09:22.881553+03:00,False,mirina83@list.ru,Новый,Пользователь,mirina83@list.ru,False,True,2022-01-18 12:48:00+03:00,,0,+79234567890,,customer,2022-12-29 20:15:07.054717+03:00,10737418240
1,5,vvvv032969,NaT,True,admin1,Администратор,Майоров,alermar96@yandex.ru,False,True,2022-01-20 11:08:51+03:00,,0,,,admin,NaT,10737418240
2,1,pbkdf2_sha256$216000$Q65DprpVn1il$4Sjfv3TrdTA8...,2023-01-16 16:48:44.716429+03:00,True,admin,Новый,Пользователь,alermar69@mail.ru,True,True,2021-10-28 18:09:54+03:00,,0,,,designer,2023-01-17 23:17:16.900045+03:00,10737418240
3,22,pbkdf2_sha256$216000$lAuZKatvyKs2$JfzSmGlDHMUg...,2023-02-02 22:37:59.134711+03:00,True,mirina1783@gmail.com,Новый,Пользователь,mirina1783@gmail.com,True,True,2023-01-12 21:33:04+03:00,,0,,,admin,2023-02-12 19:23:59.066624+03:00,10737418240
4,21,pbkdf2_sha256$216000$gxe9bTfliRwA$sfpuliudmICY...,NaT,False,alermar6917@gmail.com,Новый,Пользователь,alermar6917@gmail.com,False,True,2023-01-12 21:28:49+03:00,,0,,,designer,NaT,10737418240
5,3,pbkdf2_sha256$216000$HN2uvp4bqbRB$JE9qKcN4m5Xe...,2023-01-03 00:01:38.015501+03:00,True,alex,Новый,Пользователь,alermar17@gmail.com,True,True,2021-11-10 16:49:30+03:00,,0,,,admin,2023-01-14 00:50:00.867003+03:00,10737418240


In [55]:
import time

In [56]:
# текущий год месяц день
year = time.strftime("%Y")
month = time.strftime("%m")
day = time.strftime("%d")
year, month, day

('2023', '01', '23')

In [58]:
year_month_day = time.localtime(time.time())
year = year_month_day[0]
month = year_month_day[1]
day = year_month_day[2]
year, month, day

(2023, 1, 23)

In [57]:
time.sleep(5)

In [8]:
connection = psycopg2.connect(
            host='35.184.100.252',
            user='postgres',
            password='jmgXK4xPXukY',
            database='data_db'
        )
connection.autocommit = True

cursor = connection.cursor()

OperationalError: connection to server at "35.184.100.252", port 5432 failed: Connection timed out (0x0000274C/10060)
	Is the server running on that host and accepting TCP/IP connections?


In [ ]:
year=2021
month=12
#day=7
symbol_num=1

katalog=f'{year}_{list_month[month]}'
path = fr"G:\Общие диски\Ci\data_db_csv\{katalog}"
# путь относительно текущего скрипта
#os.mkdir("hello")
# абсолютный путь
try:
    os.mkdir(path)
except:
    pass
#os.mkdir("c://somedir/hello")

while symbol_num<100:
    try:
            # connect to exist database
        connection = psycopg2.connect(
            host='35.184.100.252',
            user='postgres',
            password='jmgXK4xPXukY',
            database='data_db'
        )
        connection.autocommit = True

        cursor = connection.cursor()

        cursor.execute(f'''SELECT datetimesecond, symbolnum, open_bid, high_bid, low_bid, close_bid, open_ask, high_ask, low_ask, close_ask
        FROM mt_input_history.inputints1
        where symbolnum={symbol_num}
        and EXTRACT(YEAR FROM datetimesecond)={year}
        and EXTRACT(month FROM datetimesecond)={month}
        order by id asc''')

        date_select = cursor.fetchall()
        my_array = np.asarray (date_select)
        df = pd.DataFrame(my_array, columns = ['datetimesecond','symbolnum','open_bid', 'high_bid', 'low_bid', 'close_bid',
                                               'open_ask', 'high_ask', 'low_ask', 'close_ask'])
        df = df.drop('symbolnum', axis=1)



        save_as(df, symbol_num, year, month)
    except Exception as _ex:
        print("[INFO] Error while working with PostgreSQL", _ex)
    finally:
        if connection:
            # cursor.close()
            connection.close()
            print("[INFO] PostgreSQL connection closed")

    symbol_num=symbol_num+1